# Proof of Concept for randomised route generation

## Items

* Load a walking networkf graph from a place using OSMNX
* Function to create a randomised route with equal interval timestamp points 
* Function to create multiple randomised routes
* Visualisation in Kepler.gl

### Load modules

In [ ]:
import osmnx as ox
from shapely.geometry import LineString
import geopandas as gpd
import numpy as np
import keplergl
import random
import pandas as pd

### Query a place and get the graph network

In [ ]:
query = 'City of London'

G = ox.graph.graph_from_place(query,
                             network_type='walk',
                             simplify=True
                             )
Gp = ox.project_graph(G, to_crs='4326')

### Function to make a randomised walking route with a fixed start time

In [ ]:
def make_route(Gp, walk_speed, start_time):

    origin_node = list(Gp.nodes)[random.randint(0, len(list(Gp.nodes)))]
    destination_node = list(Gp.nodes)[random.randint(0, len(list(Gp.nodes)))]

    route = ox.distance.shortest_path(Gp, origin_node, destination_node, weight='length', cpus=1)

    nodes = ox.graph_to_gdfs(G, nodes=True, edges=False)
    route_nodes = nodes.loc[route]

    gdf = gpd.GeoDataFrame(route_nodes, geometry='geometry', crs=4326)
    gdf = gdf.to_crs(epsg=3857)

    route_line = LineString(gdf['geometry'].tolist())
    distances = np.arange(0, route_line.length, walk_speed)

    points = [route_line.interpolate(distance) for distance in distances]

    gdf = gpd.GeoDataFrame(geometry=points, crs=3857)

    gdf['time'] = pd.date_range(start_time, freq='30s', periods=len(gdf))

    return gdf

### Function to make a fixed walking route with a fixed start time

In [ ]:
def make_route(Gp, origin_lat, origin_lon, dest_lat, dest_lon, walk_speed, start_time):

    origin_node = ox.nearest_nodes(Gp, origin_lat, origin_lon)
    destination_node = ox.nearest_nodes(Gp, dest_lat, dest_lon)

    route = ox.distance.shortest_path(Gp, origin_node, destination_node, weight='length', cpus=1)

    nodes = ox.graph_to_gdfs(G, nodes=True, edges=False)
    route_nodes = nodes.loc[route]

    gdf = gpd.GeoDataFrame(route_nodes, geometry='geometry', crs=4326)
    gdf = gdf.to_crs(epsg=3857)

    route_line = LineString(gdf['geometry'].tolist())
    distances = np.arange(0, route_line.length, walk_speed)

    points = [route_line.interpolate(distance) for distance in distances]

    gdf = gpd.GeoDataFrame(geometry=points, crs=3857)

    gdf['time'] = pd.date_range(start_time, freq='1s', periods=len(gdf))

    return gdf

### Function to make multiple routes

In [ ]:
def more_routes(Gp, total_routes, min_time, max_time, walk_speed):
    df = pd.DataFrame()
    for i in range(total_routes):
        min_time = pd.to_datetime(min_time)
        max_time = pd.to_datetime(max_time)
        random_date = min_time + (max_time - min_time) * random.random()

        route = make_route(Gp, walk_speed, random_date)
        route['ID'] = i + 1
        df = pd.concat([df, route], axis = 0)

    return df

### Generating route(s)

In [ ]:
# To make a single randomised route
df = make_route(Gp, 1.4, '2015-02-26 21:42:53')

# To make multiple randomised routes
df = more_routes(Gp, 3, '2015-02-26 21:00:00', '2015-02-26 21:30:00', 2)

### Visualising results

In [ ]:
map_1 = keplergl.KeplerGl(height=600)
map_1.add_data(data=df, name='Points')

map_1

In [ ]:
route_1 = make_route(Gp, -0.437495, 52.044001, -0.480065, 52.017105, 1.4, '2015-02-26 21:42:53')
route_2 = make_route(Gp, -0.476533, 52.031457, -0.468984, 52.034308, 1.4, '2015-02-26 21:33:53')